## This notebook contains:

### Problem 1 - Data Modeling
### Problem 2 - Ingestion
### Problem 3 - Data Analysis

In [89]:
!pip install mysql-connector-python

## Problem 1 - Data Modeling

## Creating the reqiured tables

In [20]:
import mysql.connector as connection

#Mysql connector (Python driver for communicating with MySQL servers)
try:
    mydb = connection.connect(host="localhost", database = 'Corteva',user="root", passwd="password@mysql",use_pure=True)
    # check if the connection is established
    print(mydb.is_connected())

    query = "CREATE TABLE Weather(Stationid VARCHAR(20) ,date DATE, \
            max_temperature INT(10), min_temperature INT(10),precipitation INT(10))"

    cursor = mydb.cursor() #create a cursor to execute queries
    cursor.execute(query)
    print("Table Created!!")
    mydb.close()
except Exception as e:
    mydb.close()
    print(str(e))

True
1050 (42S01): Table 'weather' already exists


In [21]:
import mysql.connector as connection

try:
    mydb = connection.connect(host="localhost", database = 'Corteva',user="root", passwd="password@mysql",use_pure=True)
    # check if the connection is established
    print(mydb.is_connected())

    query = "CREATE TABLE Yield(year INT(4),corn_yield INT(10))"

    cursor = mydb.cursor() #create a cursor to execute queries
    cursor.execute(query)
    print("Table Created!!")
    mydb.close()
except Exception as e:
    mydb.close()
    print(str(e))

True
1050 (42S01): Table 'yield' already exists


In [22]:
import mysql.connector as connection

try:
    mydb = connection.connect(host="localhost", database = 'Corteva',user="root", passwd="password@mysql",use_pure=True)
    # check if the connection is established
    print(mydb.is_connected())

    query = "CREATE TABLE Logs(Stationid VARCHAR(20), start_time TIMESTAMP, end_time TIMESTAMP,\
    num_records INT(20))"

    cursor = mydb.cursor() #create a cursor to execute queries
    cursor.execute(query)
    print("Table Created!!")
    mydb.close()
except Exception as e:
    mydb.close()
    print(str(e))

True
1050 (42S01): Table 'logs' already exists


In [23]:
import mysql.connector as connection

try:
    mydb = connection.connect(host="localhost", database = 'Corteva',user="root", passwd="password@mysql",use_pure=True)
    # check if the connection is established
    print(mydb.is_connected())

    query = "CREATE TABLE Statistics(year INT(4), Stationid VARCHAR(20), Average_maximum_temperature DECIMAL(20,2),\
    Average_minimum_temperature DECIMAL(20,2), Total_accumulated_precipitation DECIMAL(20,2))"

    cursor = mydb.cursor() #create a cursor to execute queries
    cursor.execute(query)
    print("Table Created!!")
    mydb.close()
except Exception as e:
    mydb.close()
    print(str(e))

True
1050 (42S01): Table 'statistics' already exists


In [24]:
import mysql.connector as connection

try:
    mydb = connection.connect(host="localhost", database = 'Corteva',user="root", passwd="password@mysql",use_pure=True)
    # check if the connection is established
    print(mydb.is_connected())

    query = "CREATE TABLE tmp(Stationid VARCHAR(20) ,date DATE, \
            max_temperature INT(10), min_temperature INT(10),precipitation INT(10))"

    cursor = mydb.cursor() #create a cursor to execute queries
    cursor.execute(query)
    print("Table Created!!")
    mydb.close()
except Exception as e:
    mydb.close()
    print(str(e))

True
1050 (42S01): Table 'tmp' already exists


## SHOW Tables

In [25]:
import mysql.connector as connection

try:
    mydb = connection.connect(host="127.0.0.1", database = 'Corteva',user="root", passwd="password@mysql",use_pure=True)
    # check if the connection is established

    query = "SHOW TABLES"

    cursor = mydb.cursor() #create a cursor to execute queries
    cursor.execute(query)
    print(cursor.fetchall())

except Exception as e:
    mydb.close()
    print(str(e))

[('Logs',), ('Statistics',), ('tmp',), ('Weather',), ('Yield',)]


While I had written code for comparing the duplicate records by comparing new each record with the available Weather
table it turned out to be time intensive. So had came up with the assumption the following assumptions:

1) check if the StationId already exists in the Weather table, if not exists then Insert the data into Weather Table

2) If the StationId already present in the table store the Count of records with that stationId, if this doesnt match 
to the number of records in the new file, then Delete the StationId specific specific rows from the Weather Table and 
Insert the complete new records to the Table else Igonore the new records

After considering the time required had to move with the above assumptions.

## Problem 2 - Data Ingestion

In [26]:
import os
import pandas as pd


def current_time_stamp():
    
    query = "SELECT CURRENT_TIMESTAMP"
    
    cursor = mydb.cursor() #create a cursor to execute queries
    cursor.execute(query)
    time = cursor.fetchall()[0][0]
    return time
    

directory = 'wx_data'

for filename in os.listdir(directory):

    start_time = current_time_stamp()
    
    station=filename[:-4]
    query = "SELECT EXISTS (SELECT * FROM Weather where Stationid=(%s))"

    cursor = mydb.cursor() #create a cursor to execute queries
    cursor.execute(query, (station,))
    
    weather_data_tmp = pd.read_csv('wx_data/'+filename, \
                               names=['date','max_temperature','min_temperature','precipitation'],\
                               index_col=False, delimiter = '\t', header=None)
    weather_data_tmp['Station_id']=station
    weather_data_tmp= weather_data_tmp[['Station_id','date','max_temperature','min_temperature','precipitation' ]]
    number_records= weather_data_tmp.shape[0]
    
    if cursor.fetchall()[0][0]== 0:

        # creating column list for insertion 
        cols = "','".join([str(i) for i in weather_data_tmp.columns.tolist()])
        # Insert DataFrame records one by one. 
        for i,row in weather_data_tmp.iterrows():

            query = "INSERT INTO Corteva.Weather VALUES (%s,%s,%s,%s,%s)"
            cursor.execute(query, tuple(row))
             
        end_time = current_time_stamp()
        
        query = "INSERT INTO Corteva.Logs VALUES (%s,%s,%s,%s)" 
        cursor.execute(query, (station, start_time, end_time,number_records))
    
    else:
        query = "SELECT COUNT(*) from Weather where stationid=(%s)" #number of records with given station_id

        cursor = mydb.cursor() #create a cursor to execute queries
        cursor.execute(query, (station,))
        if cursor.fetchall()[0][0]!= number_records:
            
            query = "DELETE from Corteva.Weather Where stationid=(%s)"
            cursor = mydb.cursor()
            cursor.execute(query, (station,)) 
            
            for i,row in weather_data_tmp.iterrows():
            
                query = "INSERT INTO Corteva.Weather VALUES (%s,%s,%s,%s,%s)"
                cursor = mydb.cursor()
                cursor.execute(query, tuple(row))
                
            end_time = current_time_stamp()
        
            query = "INSERT INTO Corteva.Logs VALUES (%s,%s,%s,%s)" 
            cursor.execute(query, (station, start_time, end_time,number_records))
    

In [27]:
mydb.commit()#Commit the state of Database

The following code for comparing the duplicate records by comparing new each record with the available Weather table,
had to ignore as this is time intensive.

In [28]:
# import os
# import pandas as pd

# #Function that returns the current timestamp
# def current_time_stamp():
    
#     query = "SELECT CURRENT_TIMESTAMP"
    
#     cursor = mydb.cursor() #create a cursor to execute queries
#     cursor.execute(query)
#     time = cursor.fetchall()[0][0]
#     return time
    

# directory = 'wx_data'

# for filename in os.listdir(directory):

#     start_time = current_time_stamp()
    
#     station=filename[:-4] #USC00110072.txt -> USC00110072 
    
#     #Read each file as csv and store it in weather_data_tmp dataframe 
#     weather_data_tmp = pd.read_csv('wx_data/'+filename, \
#                                names=['date','max_temperature','min_temperature','precipitation'],\
#                                index_col=False, delimiter = '\t', header=None)
    
#     #Storing the Station_id as a new column as it is required
#     weather_data_tmp['Station_id']=station
    
#     #Re-ordering the values accordingly
#     weather_data_tmp= weather_data_tmp[['Station_id','date','max_temperature','min_temperature','precipitation' ]]

#     number_records= 0    #To store the number of records

#     # creating column list for insertion 
#     cols = "','".join([str(i) for i in weather_data_tmp.columns.tolist()])
    
#     # Insert DataFrame records one by one. 
#     for i,row in weather_data_tmp.iterrows():
# #         print(row[0], row[1],sep=',')

#         #Query to check if the records already exists in the table.
#         query = "SELECT EXISTS (SELECT * FROM Weather where Stationid=%s AND date=%s)"

#         cursor = mydb.cursor() #create a cursor to execute queries
#         cursor.execute(query, (row[0], row[1]))

#         if (cursor.fetchall()[0][0]== 0):      #Record does not exist in the Table Weather so insert it
#             query = "INSERT INTO Corteva.Weather VALUES (%s,%s,%s,%s,%s)"
#             cursor.execute(query, tuple(row)) 
#             number_records+=1                #if record not exists in the table insert it and increase the counter

#     end_time = current_time_stamp()

#     query = "INSERT INTO Corteva.Logs VALUES (%s,%s,%s,%s)" 
#     cursor.execute(query, (station, start_time, end_time,number_records)) 

In [34]:
#Ingest Corn Yield data in Yield Table

import pandas as pd
corn_yield_data = pd.read_csv('yld_data/US_corn_grain_yield.txt', names=['year','corn_yield'],\
                      index_col=False, delimiter = '\t', header=None)
# creating column list for insertion 
cols = "','".join([str(i) for i in corn_yield_data.columns.tolist()])
# Insert DataFrame records one by one. 
for i,row in corn_yield_data.iterrows():

    sql = "INSERT INTO Corteva.Yield VALUES (%s,%s)"
    cursor.execute(sql, tuple(row)) 
mydb.commit()

If the incoming year is not present in the Yield table ingest the record

In [39]:
#Ingest Corn Yield data in Yield Table

import pandas as pd
corn_yield_data = pd.read_csv('yld_data/US_corn_grain_yield.txt', names=['year','corn_yield'],\
                      index_col=False, delimiter = '\t', header=None)

# creating column list for insertion 
cols = "','".join([str(i) for i in corn_yield_data.columns.tolist()])
# Insert DataFrame records one by one. 
for i,row in corn_yield_data.iterrows():
    
    query = "SELECT EXISTS (SELECT * FROM Yield where year=%s)"

    cursor = mydb.cursor() #create a cursor to execute queries
    cursor.execute(query, (int(row[0]),))

    if (cursor.fetchall()[0][0]== 0):   

        sql = "INSERT INTO Corteva.Yield VALUES (%s,%s)"
        cursor.execute(sql, tuple(row)) 
mydb.commit()

## Problem 3 - Data Analysis

In [41]:
#Calculating the statistics and storing it in the Statistics table
#Since missing values(-9999) should be ignored, incase of finding the average the Sum of all the non missing values 
#is considered in numerator and in denominator the missing values count is ignored and the query is group by year and 
#stationId.

mydb = connection.connect(host="127.0.0.1", database = 'Corteva',user="root", passwd="password@mysql",use_pure=True)

query = """SELECT YEAR(date) AS year, Stationid AS Stationid, SUM(CASE WHEN max_temperature=-9999 THEN 0 ELSE max_temperature*0.1 END)/SUM(CASE WHEN max_temperature=-9999 THEN 0 ELSE 1 END)
AS Average_maximum_temperature,
SUM(CASE WHEN min_temperature=-9999 THEN 0 ELSE min_temperature*0.1 END)/SUM(CASE WHEN min_temperature=-9999 THEN 0 ELSE 1 END)
AS Average_minimum_temperature,
SUM(CASE WHEN precipitation=-9999 THEN 0 ELSE precipitation*0.01 END) AS Total_accumulated_precipitation
FROM Weather 
GROUP BY YEAR(date), Stationid
"""

cursor = mydb.cursor() #create a cursor to execute queries
cursor.execute(query)
# print(cursor.fetchall())

lst= cursor.fetchall()

for i,row in enumerate(lst):
    sql = "INSERT INTO Corteva.Statistics VALUES (%s,%s,%s,%s,%s)"
    cursor.execute(sql, tuple(row)) 
mydb.commit()




In [42]:
lst

[(1985,
  'USC00110072',
  Decimal('15.33479'),
  Decimal('4.32645'),
  Decimal('78.01')),
 (1985,
  'USC00110187',
  Decimal('19.26537'),
  Decimal('7.81053'),
  Decimal('146.55')),
 (1985,
  'USC00110338',
  Decimal('14.42521'),
  Decimal('2.97068'),
  Decimal('95.16')),
 (1985,
  'USC00111280',
  Decimal('16.98000'),
  Decimal('6.51315'),
  Decimal('114.02')),
 (1985,
  'USC00111436',
  Decimal('16.73562'),
  Decimal('6.44137'),
  Decimal('124.63')),
 (1985,
  'USC00112140',
  Decimal('17.05753'),
  Decimal('5.20493'),
  Decimal('116.69')),
 (1985,
  'USC00112193',
  Decimal('19.20403'),
  Decimal('6.69670'),
  Decimal('79.85')),
 (1985,
  'USC00112348',
  Decimal('14.80466'),
  Decimal('1.30959'),
  Decimal('103.11')),
 (1985,
  'USC00112483',
  Decimal('18.54329'),
  Decimal('6.97068'),
  Decimal('132.09')),
 (1985,
  'USC00113335',
  Decimal('14.43699'),
  Decimal('3.35014'),
  Decimal('100.62')),
 (1985,
  'USC00113879',
  Decimal('19.36904'),
  Decimal('7.48219'),
  Decimal('14